In [133]:
import numpy as np
import numpy.random as random
import statsmodels.api as sm
import statsmodels.sandbox.regression.gmm ## For 2SLS
IV2SLS = statsmodels.sandbox.regression.gmm.IV2SLS

In [74]:
class EndogeneousDataGenerator():
    """ Endogeneous Data Generator
    
    This generates data from a data generating process that may include endogeneity
    and may (or may not) have valid instrumental variables.
    
    The data is generated from the following formulas    
    Y = outcomeFunc(T, X, U)
    T = treatmentFunc(X, Z, U)
    
    X ~ xGenerator(N)
    Z ~ zGenerator(N,X)
    U ~ uGenerator(N,X,Z)
    
    The goal is to learn the true causal effect of the potentially endogeneous treatment variable(s) T.
    The X represent known exogeneous variables which may affect the treatment assignment and the outcome Y.
    The U represent unobserved variables which may affect both the treatment assignment and the outcome
      and therefore are a potential cause of endogeneity.
    """
    def __init__(self, outcomeFunc, treatmentFunc, xGenerator, zGenerator, uGenerator):
        self.outcomeFunc = outcomeFunc
        self.treatmentFunc = treatmentFunc
        self.xGenerator = xGenerator
        self.zGenerator = zGenerator
        self.uGenerator = uGenerator

    def generateData(self, N):
        X = self.xGenerator(N)
        Z = self.zGenerator(N,X)
        U = self.uGenerator(N,X,Z)
        
        T = self.treatmentFunc(X,Z,U)
        Y = self.outcomeFunc(T,X,U)
        
        return Y, T, X, Z, U
    
    def evaluate(self, T, X, U):
        """ Returns the true outcome Y given 
        the treatment T and observables X and the unobserved values U
        """
        return self.outcomeFunc(T,X,U)


In [235]:
## Y = Wages
## X = Years Experience
## T = Years of Education
## Z = birth quarter
## U = Unobserved Skill

def treatmentFunc(X, Z, U):
    return 0.5*X + 20*Z + 8*U + random.normal(0,1,X.shape)

def outcomeFunc(T, X, U):
    return 10*T + 5*X + 6*U + random.normal(0,1,X.shape)

def xGenerator(N):
    return random.normal(10, 3, (N,1))

def zGenerator(N, X):
    return random.rand(N,1)

def uGenerator(N, X, Z):
    return random.rand(N,1)


In [236]:
dgp = EndogeneousDataGenerator(outcomeFunc, treatmentFunc, xGenerator, zGenerator, uGenerator)

In [237]:
Y, T, X, Z, U = dgp.generateData(50)
TX = sm.add_constant(np.hstack((T,X)))
XZ = sm.add_constant(np.hstack((X,Z)))

In [238]:
fittedModel = IV2SLS(Y,TX,XZ).fit()

In [239]:
fittedModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                      y   R-squared:                       0.999
Model:                         IV2SLS   Adj. R-squared:                  0.999
Method:                     Two Stage   F-statistic:                 2.768e+04
                        Least Squares   Prob (F-statistic):           6.62e-73
Date:                Sat, 15 Jul 2017                                         
Time:                        23:59:12                                         
No. Observations:                  50                                         
Df Residuals:                      47                                         
Df Model:                           2                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7966      1.140      0.699      0.488      -1.496       3.089
x1            10.0327      0.049    206.005      0.000       9.935      10.131
x2             5.1676      0.082     63.165      0.000       5.003       5.332
==============================================================================
Omnibus:                        1.239   Durbin-Watson:                   2.234
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.148
Skew:                           0.214   Prob(JB):                        0.563
Kurtosis:                       2.393   Cond. No.                         91.6
==============================================================================
"""

In [240]:
np.sum((fittedModel.predict(TX) - Y.ravel())**2)

175.41195727153445

In [241]:
np.sum((fittedModel.resid)**2)

175.41195727153445

In [242]:
TXZ = sm.add_constant(np.hstack((T,X,Z)))
fittedModel = sm.OLS(Y,TXZ).fit()

In [243]:
fittedModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.112e+04
Date:                Sat, 15 Jul 2017   Prob (F-statistic):           5.12e-81
Time:                        23:59:15   Log-Likelihood:                -79.997
No. Observations:                  50   AIC:                             168.0
Df Residuals:                      46   BIC:                             175.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0575      0.718      0.080      0.937      -1.388       1.503
x1            10.6301      0.073    144.964      0.000      10.482      10.778
x2             4.7116      0.077     61.210      0.000       4.557       4.867
x3           -12.4101      1.658     -7.485      0.000     -15.747      -9.073
==============================================================================
Omnibus:                        1.559   Durbin-Watson:                   1.810
Prob(Omnibus):                  0.459   Jarque-Bera (JB):                1.516
Skew:                           0.391   Prob(JB):                        0.469
Kurtosis:                       2.660   Cond. No.                         214.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [224]:
np.sum((fittedModel.predict(TXZ) - Y.ravel())**2)

57.651141564285268

In [225]:
np.sum(fittedModel.resid**2)

57.651141564285268